In [2]:
import sys
import re 
import numpy as np 
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from keras.utils import np_utils
import play_midi as play
from music21 import converter, instrument, note, chord, stream

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
songs = glob('midi/*.mid')
songs

['midi/MidiShow小酒窝.mid',
 'midi/MidiShow一千年以后.mid',
 'midi/MidiShow江南.mid',
 'midi/可惜没如果.mid',
 'midi/MidiShow修炼爱情.mid']

In [ ]:
stream = converter.parse('midi/可惜没如果.mid')
# 获取乐器
print('parts:')
parts = instrument.partitionByInstrument(stream)
# 输出所有乐器
for i in parts:
    print(i)

In [6]:
for index, value in enumerate(parts):
    if str(value) == "<music21.stream.Part Piano>":
        break
print(index, value)

0 <music21.stream.Part Piano>


In [34]:
from tqdm import tqdm

notes = []
for file in tqdm(songs):
    midi = converter.parse(file)  # 将.mid文件转化为流对象
    notes_to_parse = None
    try:
        # Given a single stream, partition into a part for each unique instrument
        parts = instrument.partitionByInstrument(midi)
    except:
        print("乐器分配错误!")

    if parts: # 如果parts部分有乐器
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
        print("parts == None")

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))  # 如果element是一个音符，提取其音高；音符形式如：G4、F#4
        elif(isinstance(element, chord.Chord)):
            # 如果是一个和弦, 将和弦的正常形式(整数list)添加到音符的list
            notes.append('.'.join(str(n) for n in element.normalOrder))   # 和弦形式如：'4.5.7'、'7.11'

with open('notes.pkl', 'wb') as filepath:  # 保存到文件中
    pickle.dump(notes, filepath)
    
print("notes:", notes)

100%|██████████| 4/4 [00:01<00:00,  3.10it/s]

notes: ['E4', 'C3', 'G3', 'C4', 'E4', 'F4', 'E4', 'D4', 'B2', 'G3', 'B3', 'D4', 'G4', 'D4', 'C4', 'A2', 'E3', 'A3', 'C4', 'A4', 'G4', 'E4', 'F2', 'C3', 'F3', 'F4', 'E4', 'D4', 'G2', 'D3', 'G3', 'E4', 'C3', 'G3', 'C4', 'E4', 'F4', 'G4', 'E4', 'G4', 'G2', 'D3', 'G3', 'G4', 'D5', 'B4', 'C5', 'A2', 'E3', 'A3', 'E3', 'C5', 'G2', 'B4', 'D3', 'A4', 'G3', 'B4', 'D3', 'C5', 'F2', 'C3', 'F3', 'B4', 'C3', 'C5', 'C2', 'G2', 'C3', 'G2', 'C5', 'A2', 'B4', 'E3', 'C5', 'A3', 'B4', 'C5', 'E4', 'G4', 'G2', 'D3', 'G3', 'D3', 'B3', 'D3', 'A4', 'G3', 'B4', 'D3', 'C5', 'F2', 'B4', 'C3', 'C5', 'F3', 'B4', 'C3', 'C5', 'A3', 'C3', 'D5', 'F3', 'E5', 'C3', 'C3', 'G3', 'C4', 'G3', 'E4', 'G3', 'A4', 'C4', 'B4', 'G3', 'C5', 'A2', 'B4', 'E3', 'C5', 'A3', 'B4', 'C5', 'E5', 'B4', 'G2', 'A4', 'D3', 'B4', 'G3', 'A4', 'B4', 'G4', 'C5', 'F2', 'C3', 'F3', 'C3', 'E5', 'A3', 'C3', 'F5', 'F3', 'E5', 'C3', 'D5', 'G2', 'D3', 'G3', 'A3', 'B3', '7', '0', '2', 'E5', 'C3', 'G3', 'C5', 'C4', 'G3', 'G4', 'E4', 'G3', 'G4', 'C4', 'E5',

In [4]:
sequence_length = 100

# 从音符列表中提取独特的音高并排序
pitchnames = sorted(set(item for item in notes))   # ['0', '0.3', '0.3.7', ... ]
# 对独特的音高建立整数编码
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))  # {'0': 0, '0.3': 1, '0.3.7': 2, ... ]

In [5]:
len(pitchnames)

79

In [6]:
# 创造输入和对应的输出序列
network_input = []
network_output = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i: i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])  # 转换为音符编码
    network_output.append(note_to_int[sequence_out])

In [7]:
print("network_input shape (list):", (len(network_input), len(network_input[0])))
print("network_output:", len(network_output))

network_input shape (list): (1630, 100)
network_output: 1630


In [8]:
n_patterns = len(network_input)  # 632
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))  # 将输入数据转化为LSTM输入格式, (632, 100, 1)

n_vocab = len(set(notes))
print('unique notes length:', n_vocab)
network_input = network_input / float(n_vocab)  # 归一化输入

# one hot encode the output vectors
network_output = np_utils.to_categorical(network_output)  # 输出进行

unique notes length: 79


In [9]:
network_output.shape

(1630, 79)

In [35]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten

model = Sequential()
model.add(LSTM(128, input_shape=network_input.shape[1:], return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

"""
另一个模型
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
"""

In [64]:
from keras.callbacks import ModelCheckpoint

epochs = 100
filepath = 'weights.music4.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])

Epoch 1/100
1630/1630 [==============================] - 14s 8ms/step - loss: 0.0310
Epoch 2/100
1630/1630 [==============================] - 17s 11ms/step - loss: 0.0363
Epoch 3/100
1630/1630 [==============================] - 17s 10ms/step - loss: 0.0411
Epoch 4/100
1630/1630 [==============================] - 17s 11ms/step - loss: 0.0440
Epoch 5/100
1630/1630 [==============================] - 17s 10ms/step - loss: 0.0430
Epoch 6/100
1630/1630 [==============================] - 17s 11ms/step - loss: 0.0588
Epoch 7/100
1630/1630 [==============================] - 17s 10ms/step - loss: 0.0459
Epoch 8/100
1630/1630 [==============================] - 17s 11ms/step - loss: 0.0381
Epoch 9/100
1630/1630 [==============================] - 17s 11ms/step - loss: 0.0751
Epoch 10/100
1630/1630 [==============================] - 17s 10ms/step - loss: 0.0529
Epoch 11/100
1630/1630 [==============================] - 17s 10ms/step - loss: 0.0396
Epoch 12/100
1630/1630 [=============================

## 训练完成，开始生成音频

In [57]:
# 从保存的数据中加载模型
#with open('notes.pkl', 'rb') as filepath:
#    notes = pickle.load(filepath)

#print('Initiating music generation process.......')

In [65]:
network_input.shape

(1630, 100, 1)

In [66]:
start = np.random.randint(0, len(network_input)-1)  # 选择一个随机整数
print("start:", start)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

# pick a random sequence from the input as a starting point for the prediction
pattern = network_input[start]
prediction_output = []

print('Generating notes........')

start: 1495
Generating notes........


In [67]:
print("pattern.shape:", pattern.shape)
pattern[:10]

pattern.shape: (100, 1)


array([[0.93670886],
       [0.79746835],
       [0.41772152],
       [0.4556962 ],
       [0.56962025],
       [0.4556962 ],
       [0.81012658],
       [0.43037975],
       [0.59493671],
       [0.81012658]])

In [68]:
# 产生500个音符
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    predict_index = np.argmax(prediction)  # Predicted output is the argmax(P(h|D))
    result = int_to_note[predict_index]   # 返回整数标签对应的音符
    prediction_output.append(result)

    pattern = np.append(pattern, predict_index)
    # Next input to the model
    pattern = pattern[1:1+len(pattern)]

print('Notes Generated...')

Notes Generated...


In [71]:
prediction_output[:10]

['G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5']

In [72]:
# 将预测值转换为音符并写入midi文件
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():  # pattern是和弦
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()   # 全是钢琴
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:  # pattern是一个音符
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()  # 钢琴
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
print('Saving Output file as midi....')
midi_stream.write('midi', fp='data/JJLin3.mid')

Saving Output file as midi....


'data/JJLin3.mid'

In [18]:
### Play the Jazz music
# play.play_midi('./generated.mid')

File ./generated.mid not found! (/etc/timidity.cfg: No such file or directory)
